In [ ]:
# Configuration
from dotenv import dotenv_values

config = dotenv_values(".env")
graphql_endpoint = config["DRUPAL_GRAPHQL_URL"]
course_acronym = f"{config['DRUPAL_COURSE_ACRONYM']}: "

import os

json_filename = f"{config['DRUPAL_COURSE_ID']}.json"
course_json_path = os.path.join(config["COURSE_DATA_DIR"], json_filename)

In [ ]:
# Load course json
from json import load

with open(file=course_json_path, mode="r", encoding="utf-8") as f:
    course_json = load(f)

In [ ]:
from transform_course_json import apply_transformations

course_json = apply_transformations(course_json)

In [ ]:
# Authentication
from requests import session

with session() as session:
    session.auth = (config["DRUPAL_USERNAME"], config["DRUPAL_PASSWORD"])

In [ ]:
res = session.post(
    graphql_endpoint,
    json={
        "query": """query GetAllBooks {
        books {
            id,
            title,
            body {
                value,
                summary
            },
            links {
                bid
            }
        }
    }""",
    },
)
res.json()

In [ ]:
# Create new course page
res = session.post(
    graphql_endpoint,
    json={
        "query": """mutation CreateCourseBookPage($data: BookInput!) {
        createBook(data: $data) {
            errors,
            book {
                id,
                title,
                body {
                    value,
                    summary,
                    format
                }
            }
        }
    }""",
        "variables": {
            "data": {
                "title": course_acronym + course_json["course_name"],
                "body": {
                    "value": course_json["overview"],
                },
            }
        },
    },
)
new_book_id = res.json()["data"]["createBook"]["book"]["id"]
res.json()

In [ ]:
# Make the new course page into a book
res = session.post(
    graphql_endpoint,
    json={
        "query": """mutation AddCourseBookOutline($id: ID!, $data: BookInput!) {
        updateBook(id: $id, data: $data) {
            errors,
            book {
                id,
                links {
                    nid,
                    bid
                }
            }
        }
    }""",
        "variables": {
            "id": new_book_id,
            "data": {
                "title": course_acronym + course_json["course_name"],
                "links": {"bid": new_book_id},
            },
        },
    },
)
res.json()

In [ ]:
def create_outline_page(title, book_id, weight):
    res = session.post(
        graphql_endpoint,
        json={
            "query": """mutation CreateCourseOutlinePage($data: BookInput!) {
            createBook(data: $data) {
                errors,
                book {
                    id,
                    title,
                    links {
                        bid,
                        pid,
                        weight,
                    }
                }
            }
        }""",
            "variables": {
                "data": {
                    "title": course_acronym + title,
                    "links": {
                        "bid": book_id,
                        "pid": book_id,
                        "weight": weight,
                    },
                }
            },
        },
    )
    print(res.json())
    return res.json()["data"]["createBook"]["book"]["id"]


def create_content_page(title, summary, body, book_id, parent_id, weight):
    res = session.post(
        graphql_endpoint,
        json={
            "query": """mutation CreateCourseContentPage($data: BookInput!) {
            createBook(data: $data) {
                errors,
                book {
                    id,
                    title,
                    body {
                        value,
                        summary,
                        format,
                    },
                    links {
                        bid,
                        pid,
                        weight,
                    }
                }
            }
        }""",
            "variables": {
                "data": {
                    "title": course_acronym + title,
                    "body": {
                        "summary": summary,
                        "value": body,
                    },
                    "links": {
                        "bid": book_id,
                        "pid": parent_id,
                        "weight": weight,
                    },
                }
            },
        },
    )
    print(res.json())
    return res.json()["data"]["createBook"]["book"]["id"]


def create_content_pages_rec(toc_subitem, book_id, parent_id, weight):
    page_id = create_content_page(
        toc_subitem["title"],
        toc_subitem["description"],
        toc_subitem["body"],
        book_id,
        parent_id,
        weight,
    )
    for child_index, child in enumerate(toc_subitem["children"]):
        create_content_pages_rec(child, book_id, page_id, child_index)


# Add a content pages linked to the course book
for toc_index, toc_item in enumerate(course_json["toc"]):
    toc_page_id = create_outline_page(toc_item["title"], new_book_id, toc_index)
    for child_index, child in enumerate(toc_item["children"]):
        create_content_pages_rec(child, new_book_id, toc_page_id, child_index)